In [21]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Capstone-KPMG/Preprocessing

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/.shortcut-targets-by-id/1vvRy60J06fZq-dj8EPVTkgHU4gq8OCqU/Capstone-KPMG/Preprocessing


In [22]:
!ls Data/

'alt_fuel_stations(Sep_25_2022).csv'
 attraction_point.csv
 cleaned_main_data.csv
'Copy of I5Exits.csv'
 gas_station_w_10mi_to_exit_I5_1.csv
 gas_station_w_10mi_to_exit_I5_2.csv
 gas_station_w_10mi_to_exit_I5_3.csv
 gas_station_w_10mi_to_exit_I5_4.csv
 gas_station_w_10mi_to_exit_I5_5.csv
 gas_station_w_10mi_to_exit_I5_6.csv
 gas_station_w_10mi_to_exit_I82.csv
 gas_station_w_10mi_to_exit_I90_1.csv
 gas_station_w_10mi_to_exit_I90_2.csv
 gas_station_w_10mi_to_exit_whole.csv
 gas_station_w_attr.csv
 gas_station_w_crime_population.csv
 gas_station_w_dist_to_exit_all.csv
 gas_station_w_dist_to_exit_I5_1.csv
 gas_station_w_dist_to_exit_I5_2.csv
 gas_station_w_dist_to_exit_I5_3.csv
 gas_station_w_dist_to_exit_I5_4.csv
 gas_station_w_dist_to_exit_I5.csv
 gas_station_w_dist_to_exit_I82.csv
 gas_station_w_dist_to_exit_I90.csv
 gas_station_w_dist_to_exit_wholeI5.csv
 gas_station_w_dist_to_exit_wholeI5I90I82.csv
 gas_station_w_dist_to_exit_wholeI90.csv
 gas_station_w_ev_station_cnt.csv
 gas_station

In [23]:
import pandas as pd
import numpy as np

In [24]:
attr = pd.read_csv('Data/gas_station_w_attr.csv',index_col='Unnamed: 0')

In [25]:
crime = pd.read_csv('Data/gas_station_w_crime_population.csv',index_col='Unnamed: 0').drop(columns=['name'])

In [26]:
crime = crime.rename(columns={'Coord': 'crime_coord', 'County': 'crime_county', 'Population': 'crime_population', "Violent\ncrime": 'violent_crime',
                      'Murder and\nnonnegligent\nmanslaughter': 'murder_nonnegligent_manslaughter', 'Aggravated\nassault': 'aggravated_assault',
                      'Property\ncrime':'property_crime', 'Larceny-\ntheft':'larceny_theft', 'Motor\nvehicle\ntheft': 'motor_vehicle_theft', 'Total Crime': 'total_crime'})

In [27]:
whole = attr.merge(crime,left_on=['gas_lat','gas_long'],right_on = ['Latitude','Longitude']).drop(columns=['Latitude','Longitude'])

In [28]:
exits = pd.read_csv('Data/gas_station_w_10mi_to_exit_whole.csv',index_col='Unnamed: 0')
exits = exits.drop(columns=['gas_lat', 'gas_long'])

In [29]:
# whole1 = whole.merge(exits,on=['gas_lat','gas_long','gas_key'],how='left')
# whole2 = whole.merge(exits,on=['gas_key'],how='left')
# whole3 = whole.merge(exits,on=['gas_lat','gas_long'],how='left')
whole = whole.merge(exits,on=['gas_key'],how='left')

In [30]:
ev = pd.read_csv('Data/wa_gas_stations_w_ev_info.csv',index_col='Unnamed: 0')

ev = ev.rename(columns={'Closest_distance': 'distance_to_closest_ev_station'})

In [31]:
whole = whole.merge(ev,on=['gas_lat','gas_long','gas_key'],how='left')

In [32]:
nri = pd.read_csv('Data/gas_station_w_nri.csv',index_col='Unnamed: 0')
nri.columns

Index(['Longitude', 'Latitude', 'GEOID', 'COUNTY', 'POPULATION', 'BUILDVALUE',
       'AGRIVALUE', 'AREA', 'RISK_SCORE', 'RISK_RATNG', 'INTPTLAT',
       'INTPTLONG', 'zipcode', 'geometry', 'category'],
      dtype='object')

In [33]:
nri = nri.rename(columns={'GEOID': 'nri_geoid', 'COUNTY': 'nri_county', 'POPULATION': 'nri_population', "BUILDVALUE": 'nri_build_value',
                      'AGRIVALUE': 'nri_agri_value', 'AREA': 'nri_area', 'RISK_SCORE':'nri_risk_score', 'RISK_RATNG':'nri_risk_rating', 
                      'INTPTLAT': 'nri_intpt_lat', 'INTPTLONG': 'nri_intpt_long', 'zipcode': 'nri_zipcode', "geometry": "census_tract_area", 
                      "category": "census_tract_category"})

In [34]:
whole = whole.merge(nri,left_on = ['gas_lat','gas_long'],right_on = ['Latitude','Longitude'],how='left').drop(columns=['Latitude','Longitude'])

In [35]:
traffic = pd.read_csv('Data/gas_station_w_traffic.csv',index_col='Unnamed: 0')

In [36]:
whole = whole.merge(traffic,on=['gas_lat','gas_long','gas_name','gas_key'],how='left')

In [37]:
whole.columns

Index(['gas_key', 'gas_name', 'gas_lat', 'gas_long', 'attr_cnt_1mile',
       'attr_cnt_5mile', 'attr_name', 'attr_lat', 'attr_long',
       'distance_to_nearest_attr', 'crime_coord', 'crime_county',
       'crime_population', 'violent_crime', 'murder_nonnegligent_manslaughter',
       'Rape1', 'Robbery', 'aggravated_assault', 'property_crime', 'Burglary',
       'larceny_theft', 'motor_vehicle_theft', 'Arson', 'total_crime',
       'exit_name', 'exit_lat', 'exit_long', 'distance_to_nearest_exit',
       'interstate', 'num_EV_in_2_miles_of_gas', 'num_EV_in_5_miles_of_gas',
       'num_EV_in_10_miles_of_gas', 'num_EV_in_20_miles_of_gas',
       'num_EV_in_50_miles_of_gas', 'Closest_EV_Station_name',
       'Closest_EV_Station_lat', 'Closest_EV_Station_long',
       'distance_to_closest_ev_station', 'nri_geoid', 'nri_county',
       'nri_population', 'nri_build_value', 'nri_agri_value', 'nri_area',
       'nri_risk_score', 'nri_risk_rating', 'nri_intpt_lat', 'nri_intpt_long',
       'nri

In [38]:
whole = whole.rename(columns={'interstate':'highway'})

In [39]:
whole.to_csv('Data/main_dataset_all_interstate_all_route_10mi.csv',index=False)

In [41]:
whole.distance_to_nearest_exit.max()

9.7656